# Importando as Bibliotecas

In [1]:
import psycopg2
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
import joblib
from sqlalchemy import create_engine
import time
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import io

# Conectando com o Banco de Dados

In [2]:
conn = psycopg2.connect(
    host="fidelidademaisibm.cb4g0aguevt3.us-east-2.rds.amazonaws.com",
    database="ibm",
    user="thiago",
    password="ibmteste",
    port=5432
)

cursor = conn.cursor()
cursor.execute("SELECT version();")
print(cursor.fetchone())

('PostgreSQL 17.4 on aarch64-unknown-linux-gnu, compiled by gcc (GCC) 12.4.0, 64-bit',)


# Listando Todas as Tabelas e Suas Colunas

In [3]:
cursor.execute("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public'
""")
tables = cursor.fetchall()
print("Tabelas disponíveis:")
for t in tables:
    print(t[0])

Tabelas disponíveis:
ibm_rfm
ibm_familia
ibm_grupo
ibm_grupo_marca
ibm_marca
ibm_produto
ibm_itens
ibm_linha
ibm_linha_subgrupo
ibm_subgrupo
ibm_cliente
nota_fiscal
ibm_operacao
ibm_vendedor
modelos_ml
churn_features


In [4]:
# Função para listar colunas de cada tabela
def listar_colunas(conn, tabela):
    query = f"""
    SELECT column_name, data_type
    FROM information_schema.columns
    WHERE table_name = '{tabela}';
    """
    return pd.read_sql(query, conn)

# Mostrando colunas de todas as tabelas
for table in tables:
    print(f"\n📄 Estrutura da tabela: {table}")
    print(listar_colunas(conn, table[0]))


📄 Estrutura da tabela: ('ibm_rfm',)
   column_name         data_type
0   cd_cliente  double precision
1     recencia            bigint
2   frequencia            bigint
3  valor_total  double precision
4  valor_medio  double precision

📄 Estrutura da tabela: ('ibm_familia',)


C:\Users\henri\AppData\Local\Temp\ipykernel_24748\1240962458.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)
C:\Users\henri\AppData\Local\Temp\ipykernel_24748\1240962458.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)


  column_name          data_type
0  cd_familia            numeric
1  nm_familia  character varying

📄 Estrutura da tabela: ('ibm_grupo',)
  column_name          data_type
0    cd_grupo            numeric
1  cd_familia            numeric
2    nm_grupo  character varying

📄 Estrutura da tabela: ('ibm_grupo_marca',)


C:\Users\henri\AppData\Local\Temp\ipykernel_24748\1240962458.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)
C:\Users\henri\AppData\Local\Temp\ipykernel_24748\1240962458.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)


  column_name data_type
0    cd_grupo   numeric
1    cd_marca   numeric

📄 Estrutura da tabela: ('ibm_marca',)
  column_name          data_type
0    cd_marca            numeric
1    nm_marca  character varying

📄 Estrutura da tabela: ('ibm_produto',)


...

   column_name data_type
0      nr_nota   numeric
1   dt_emissao      date
2   cd_cliente   numeric
3  cd_vendedor   numeric
4  cd_operacao   numeric
5     vl_total   numeric

📄 Estrutura da tabela: ('ibm_operacao',)
   column_name          data_type
0  cd_operacao            numeric
1  tp_operacao  character varying

📄 Estrutura da tabela: ('ibm_vendedor',)


C:\Users\henri\AppData\Local\Temp\ipykernel_24748\1240962458.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)
C:\Users\henri\AppData\Local\Temp\ipykernel_24748\1240962458.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)


   column_name          data_type
0  cd_vendedor            numeric
1  nm_vendedor  character varying

📄 Estrutura da tabela: ('modelos_ml',)
  column_name                    data_type
0          id                      integer
1   criado_em  timestamp without time zone
2      modelo                        bytea
3   nm_modelo            character varying
4      versao            character varying

📄 Estrutura da tabela: ('churn_features',)


C:\Users\henri\AppData\Local\Temp\ipykernel_24748\1240962458.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)
C:\Users\henri\AppData\Local\Temp\ipykernel_24748\1240962458.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)


                  column_name data_type
0                  cd_cliente   numeric
1    dias_desde_ultima_compra   integer
2  dias_desde_primeira_compra   integer
3          frequencia_compras   integer
4         valor_total_compras   numeric
5                ticket_medio   numeric
6      qtd_produtos_distintos   integer
7      media_itens_por_compra   numeric
8               data_execucao      date


# Importando os Dados

In [5]:
query = """
WITH base_notas AS (
    SELECT
        nf.cd_cliente,
        nf.dt_emissao,
        nf.vl_total,
        i.cd_produto,
        i.qtd_itens
    FROM nota_fiscal nf
    LEFT JOIN ibm_itens i ON nf.nr_nota = i.nr_nota
),

agregado_cliente AS (
    SELECT
        cd_cliente,
        CURRENT_DATE - MAX(dt_emissao) AS dias_desde_ultima_compra,
        CURRENT_DATE - MIN(dt_emissao) AS dias_desde_primeira_compra,
        COUNT(DISTINCT dt_emissao) AS frequencia_compras,
        SUM(vl_total) AS valor_total_compras,
        AVG(vl_total) AS ticket_medio,
        COUNT(DISTINCT cd_produto) AS qtd_produtos_distintos,
        SUM(qtd_itens) * 1.0 / COUNT(DISTINCT dt_emissao) AS media_itens_por_compra
    FROM base_notas
    GROUP BY cd_cliente
)

SELECT *
FROM agregado_cliente;
"""
df_features = pd.read_sql(query, conn)
df_features.head()

C:\Users\henri\AppData\Local\Temp\ipykernel_24748\3699468532.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_features = pd.read_sql(query, conn)


,cd_cliente,dias_desde_ultima_compra,dias_desde_primeira_compra,frequencia_compras,valor_total_compras,ticket_medio,qtd_produtos_distintos,media_itens_por_compra
0,1.0,4870,6719,8,12195.20,1524.400000,2,1.125000
1,2.0,5858,43216,7,9867.20,1409.600000,1,1.000000
2,3.0,5398,6684,16,29952.80,1761.929412,6,1.250000
3,4.0,5301,6680,307,638150.72,1927.947795,71,1.462541
4,5.0,5855,6677,3,3601.76,900.440000,3,1.333333


In [6]:
query = """
SELECT 
    r.cd_cliente,
    r.recencia,
    r.frequencia,
    r.valor_total,
    r.valor_medio,
    c.uf_estado,
    CASE 
        WHEN r.recencia > 500 THEN 1
        ELSE 0
    END AS churn
FROM ibm_rfm r
JOIN ibm_cliente c ON r.cd_cliente = c.cd_cliente;
"""

data = pd.read_sql(query, conn)
conn.close()

C:\Users\henri\AppData\Local\Temp\ipykernel_24748\3700148924.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, conn)


# Separação dos dados

In [7]:
df_churn = data[['cd_cliente', 'churn']]
df_modelo = df_features.merge(df_churn, on='cd_cliente')

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   cd_cliente   270 non-null    float64
 1   recencia     270 non-null    int64  
 2   frequencia   270 non-null    int64  
 3   valor_total  270 non-null    float64
 4   valor_medio  270 non-null    float64
 5   uf_estado    270 non-null    object 
 6   churn        270 non-null    int64  
dtypes: float64(3), int64(3), object(1)
memory usage: 14.9+ KB


In [9]:
# Preparando as features
X = df_modelo.drop(columns=['cd_cliente', 'churn', 'dias_desde_ultima_compra'])

# Salvando o identificador e target
y = df_modelo['churn']

# Normalizando
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Separando treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42, stratify=y)
print(X.columns.tolist())


['dias_desde_primeira_compra', 'frequencia_compras', 'valor_total_compras', 'ticket_medio', 'qtd_produtos_distintos', 'media_itens_por_compra']


# Treinamento e avalição dos modelos

In [10]:
def predict(grid_search, model_type, features_train, target_train, features_valid, target_valid):
    # Obtendo os melhores parâmetros
    best_params = grid_search.best_params_

    # Selecionando o modelo com base no tipo
    if model_type == 'RandomForest':
        best_model = RandomForestClassifier(**best_params, random_state=12345)
    elif model_type == 'XGBoost':
        best_model = XGBClassifier(**best_params, random_state=12345, use_label_encoder=False, eval_metric='logloss')
    elif model_type == 'CatBoost':
        best_model = CatBoostClassifier(**best_params, random_state=12345, silent=True)
    else:
        raise ValueError("Modelo não suportado")

    # Treinando o modelo com os melhores parâmetros
    start_time = time.time()
    best_model.fit(features_train, target_train)
    end_time = time.time()
    execution_time = end_time - start_time

    # Previsões
    predicted_valid_prob = best_model.predict_proba(features_valid)[:, 1]
    predicted_valid_class = best_model.predict(features_valid)

    # Avaliações
    auc_score = roc_auc_score(target_valid, predicted_valid_prob)
    acc_score = accuracy_score(target_valid, predicted_valid_class)

    # Validação cruzada de acurácia
    cv_scores = cross_val_score(best_model, features_train, target_train, cv=5, scoring='accuracy')

    # Resultados
    print(f"\n📊 Modelo: {model_type}")
    print("Melhores parâmetros:", best_params)
    print(f"AUC-ROC (validação): {auc_score:.4f}")
    print(f"Acurácia (validação): {acc_score:.4f}")
    print(f"Acurácia (cross-val média): {cv_scores.mean():.4f}")
    print(f"Tempo de execução: {execution_time:.2f} segundos")
    print("\nRelatório de Classificação:\n")
    print(classification_report(target_valid, predicted_valid_class))    

In [11]:
unique, counts = np.unique(y_train, return_counts=True)
print(dict(zip(unique, counts)))

{0: 93, 1: 96}


## RandomForestClassifier

In [12]:
# Definindo o modelo
model1 = RandomForestClassifier(random_state=12345, class_weight='balanced')

# Definindo os parâmetros para busca
param_grid1 = {
    'n_estimators': [10, 50],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}

# Configurando a validação cruzada estratificada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Configurando a busca com validação cruzada
grid_search1 = GridSearchCV(estimator=model1, param_grid=param_grid1, cv=cv, scoring='accuracy', n_jobs=-1)

# Executando a busca
grid_search1.fit(X_train, y_train)
#, error_score='raise'

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(class_weight='balanced',
                                              random_state=12345),
             n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20],
                         'max_features': ['sqrt', 'log2', None],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [10, 50]},
             scoring='accuracy')

In [13]:
predict(grid_search1, 'RandomForest', X_train, y_train, X_test, y_test)


📊 Modelo: RandomForest
Melhores parâmetros: {'max_depth': None, 'max_features': None, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 50}
AUC-ROC (validação): 0.8570
Acurácia (validação): 0.8395
Acurácia (cross-val média): 0.7935
Tempo de execução: 0.06 segundos

Relatório de Classificação:

              precision    recall  f1-score   support

           0       0.83      0.85      0.84        40
           1       0.85      0.83      0.84        41

    accuracy                           0.84        81
   macro avg       0.84      0.84      0.84        81
weighted avg       0.84      0.84      0.84        81



## XGBClassifier

In [14]:
# Definindo o modelo
model2 = XGBClassifier(random_state=12345, use_label_encoder=False, eval_metric='logloss')

# Definindo os parâmetros para busca
param_grid2 = {
    'n_estimators': [10, 50],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.3],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Configurando a busca com validação cruzada
grid_search2 = GridSearchCV(estimator=model2, param_grid=param_grid2, cv=cv, scoring='accuracy', n_jobs=-1)

# Executando a busca
grid_search2.fit(X_train, y_train)

c:\Users\henri\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:10:35] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='logloss', feature_types=None,
                                     feature_weights=None, gamma=None,
                                     grow_...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.8, 1.0],
                         'learning_rate': [0.01, 0.1, 0.3],
                         'max_depth': [3, 5, 7], 'n_estimators': [10, 50],
                         'subsample': [0.8, 1.0]},
             scoring='accuracy')

In [15]:
predict(grid_search2, 'XGBoost', X_train, y_train, X_test, y_test)


📊 Modelo: XGBoost
Melhores parâmetros: {'colsample_bytree': 1.0, 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 10, 'subsample': 0.8}
AUC-ROC (validação): 0.8348
Acurácia (validação): 0.7901
Acurácia (cross-val média): 0.8255
Tempo de execução: 0.01 segundos

Relatório de Classificação:

              precision    recall  f1-score   support

           0       0.81      0.75      0.78        40
           1       0.77      0.83      0.80        41

    accuracy                           0.79        81
   macro avg       0.79      0.79      0.79        81
weighted avg       0.79      0.79      0.79        81



c:\Users\henri\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:10:35] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\henri\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:10:35] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\henri\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:10:35] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\henri\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:10:35] WARNING: C:\actio

In [16]:
data.corr(numeric_only=True)['churn'].sort_values(ascending=False)

churn          1.000000
recencia       0.654594
valor_medio   -0.084193
cd_cliente    -0.244489
frequencia    -0.334506
valor_total   -0.359391
Name: churn, dtype: float64

## CatBoostClassifier

In [17]:
# Definindo o modelo
model3 = CatBoostClassifier(random_state=12345, silent=True)

# Definindo os parâmetros para busca
param_grid3 = {
    'iterations': [10, 50],
    'depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.3],
    'l2_leaf_reg': [1, 3, 5],
    'border_count': [32, 50, 100]
}

# Configurando a busca com validação cruzada
grid_search3 = GridSearchCV(estimator=model3, param_grid=param_grid3, cv=cv, scoring='accuracy', n_jobs=-1)

# Executando a busca
grid_search3.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=<catboost.core.CatBoostClassifier object at 0x00000237D7557C50>,
             n_jobs=-1,
             param_grid={'border_count': [32, 50, 100], 'depth': [3, 5, 7],
                         'iterations': [10, 50], 'l2_leaf_reg': [1, 3, 5],
                         'learning_rate': [0.01, 0.1, 0.3]},
             scoring='accuracy')

In [18]:
predict(grid_search3, 'CatBoost', X_train, y_train, X_test, y_test)


📊 Modelo: CatBoost
Melhores parâmetros: {'border_count': 50, 'depth': 5, 'iterations': 10, 'l2_leaf_reg': 1, 'learning_rate': 0.3}
AUC-ROC (validação): 0.8860
Acurácia (validação): 0.8519
Acurácia (cross-val média): 0.8040
Tempo de execução: 0.06 segundos

Relatório de Classificação:

              precision    recall  f1-score   support

           0       0.85      0.85      0.85        40
           1       0.85      0.85      0.85        41

    accuracy                           0.85        81
   macro avg       0.85      0.85      0.85        81
weighted avg       0.85      0.85      0.85        81



In [19]:
#X_train = data[['recencia', 'frequencia', 'valor_monetario', 'idade', 'tempo_cliente', 'qtde_compras']]


In [20]:
model = CatBoostClassifier(
    border_count=50,
    depth=5,
    iterations=10,
    l2_leaf_reg=1,
    learning_rate=0.3,
    verbose=0  # Evita print de cada iteração
)
# Treinando o modelo
model.fit(X_train, y_train)

In [ ]:
print(model.feature_names_)

['0', '1', '2', '3', '4', '5']


# Salvando o Melhor Modelo

Modelo salvo no banco com sucesso!


# Serializando o modelo para um buffer de memória
buffer = io.BytesIO()
joblib.dump(model3, buffer)
buffer.seek(0)

# Conectando ao banco PostgreSQL
conn = psycopg2.connect(
    host="fidelidademaisibm.cb4g0aguevt3.us-east-2.rds.amazonaws.com",
    database="ibm",
    user="thiago",
    password="ibmteste",
    port=5432
)
cursor = conn.cursor()

# Inserindo o modelo
cursor.execute("""
    INSERT INTO modelos_ml (nm_modelo, versao, modelo)
    VALUES (%s, %s, %s)
""", ("catboost_churn", "v1", buffer.read()))

conn.commit()
cursor.close()
conn.close()
print("Modelo salvo no banco com sucesso!")

In [22]:
"""# Extrair o melhor modelo da busca
catboost_model = grid_search3.best_estimator_

# Salvar o modelo treinado
joblib.dump(catboost_model, "modelo_catboost.pkl")"""

'# Extrair o melhor modelo da busca\ncatboost_model = grid_search3.best_estimator_\n\n# Salvar o modelo treinado\njoblib.dump(catboost_model, "modelo_catboost.pkl")'

In [23]:
"""# Previsões no conjunto de validação
predicted_valid_prob = catboost_model.predict_proba(X_test)[:, 1]
predicted_valid_class = catboost_model.predict(X_test)

# Se tiver IDs dos clientes
df_pred = pd.DataFrame({
    'cd_cliente': df_modelo.loc[X_test.index, 'cd_cliente'].values,  # substitua pelo nome correto se necessário
    'classe_real': y_test.values,
    'classe_predita': predicted_valid_class,
    'probabilidade_churn': predicted_valid_prob
})

# Salvar em CSV
df_pred.to_csv("predicoes_catboost.csv", index=False)
print("✔️ Previsões salvas em predicoes_catboost.csv")"""

'# Previsões no conjunto de validação\npredicted_valid_prob = catboost_model.predict_proba(X_test)[:, 1]\npredicted_valid_class = catboost_model.predict(X_test)\n\n# Se tiver IDs dos clientes\ndf_pred = pd.DataFrame({\n    \'cd_cliente\': df_modelo.loc[X_test.index, \'cd_cliente\'].values,  # substitua pelo nome correto se necessário\n    \'classe_real\': y_test.values,\n    \'classe_predita\': predicted_valid_class,\n    \'probabilidade_churn\': predicted_valid_prob\n})\n\n# Salvar em CSV\ndf_pred.to_csv("predicoes_catboost.csv", index=False)\nprint("✔️ Previsões salvas em predicoes_catboost.csv")'